In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from preproc3 import preprocessing

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
gb1=GradientBoostingClassifier(n_estimators=450, learning_rate=0.14, subsample=0.9)
gb2=GradientBoostingClassifier(n_estimators=500, learning_rate=0.12, subsample=0.9)
gb3=GradientBoostingClassifier(n_estimators=550, learning_rate=0.1, subsample=0.9)
gb4=GradientBoostingClassifier(n_estimators=600, learning_rate=0.08, subsample=0.9)

lr=LogisticRegression(max_iter=1000)

estimators=[('gb1',gb1),('gb2',gb2),('gb3',gb3),('gb4',gb4)]
model=StackingClassifier(passthrough=True, estimators=estimators,final_estimator=lr,n_jobs=4,verbose=100)

In [ ]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

RESULTS='../results/'

In [ ]:
train = pd.read_csv(TRAIN) # Cargo datos de entrenamiento
test = pd.read_csv(TEST) # Cargo datos de test

# Eliminamos el campo id ya que no se debe usar para predecir
test_ids = test['id']
del test['id']
del train['id']

# Cambiamos el nombre a la columna Año para poder manejarla correctamente
train.rename(columns = {'Año':'Anio'}, inplace = True)
test.rename(columns = {'Año':'Anio'}, inplace = True)

In [ ]:
train, label, test = preprocessing(train, test)

In [ ]:
scores=cross_val_score(model, train, label, cv=5)
print(scores)
print(np.mean(scores))

In [ ]:
model.fit(train,label)
# Ahora predecimos
predict = model.predict(test)
predict = list(map(int,predict))
# Generamos 
df_result = pd.DataFrame({'id': test_ids, 'Precio_cat': predict})
df_result.to_csv(RESULTS+"try17.csv", index=False)